## GCC118 - Programação Matemática
### Universidade Federal de Lavras
---
#### Luis Kennedy Gervásio Turola
#### Thaís Giovanna Lopes
#### Tobias Maugus Bueno Cougo

---

# Problema de Otimização: Aquisição de Aeronaves

A **Empresa de Viação Aérea Brasileira** estuda a compra de três tipos de aviões:

- **Boeing 717** (curta distância)  
- **Boeing 737-500** (média distância)  
- **MD-11** (longa distância)  

Os dados de custo, receita e pilotos aptos estão na tabela:

| Avião            | Custo (milhões \$) | Receita (milhões \$) | Pilotos aptos |
|------------------|---------------------|-----------------------|---------------|
| Boeing 717       | 5.1                 | 330                   | 30            |
| Boeing 737-500   | 3.6                 | 300                   | 20            |
| MD-11            | 6.8                 | 420                   | 10            |

**Outras restrições:**
- Verba disponível: **220 milhões de \$**.  
- Cada aeronave precisa de **2 pilotos**.  
- Pilotos do MD-11 podem pilotar **qualquer aeronave**; os demais só pilotam o modelo em que são habilitados.  
- Capacidade de manutenção: **40 Boeings 717 equivalentes**.  
  - Boeing 717 = 1 unidade  
  - Boeing 737-500 = 3/4 de unidade  
  - MD-11 = 5/3 de unidade  

---

## Modelagem Matemática

### Variáveis de decisão
- $x_1$ = número de **Boeings 717** comprados.  
- $x_2$ = número de **Boeings 737-500** comprados.  
- $x_3$ = número de **MD-11** comprados.  

Para modelar a **alocação de pilotos**:  
- $p_1$ = pilotos do pool 717 usados em 717.  
- $p_2$ = pilotos do pool 737 usados em 737.  
- $p_{31}$ = pilotos do pool MD-11 usados em 717.  
- $p_{32}$ = pilotos do pool MD-11 usados em 737.  
- $p_{33}$ = pilotos do pool MD-11 usados em MD-11.  

---

### Função Objetivo
Maximizar a receita total obtida pelas aeronaves:

$$
\max Z = 330x_1 + 300x_2 + 420x_3
$$

---

### Restrições

**1. Orçamento disponível:**

$$
5.1x_1 + 3.6x_2 + 6.8x_3 \;\leq\; 220
$$

---

**2. Pilotos necessários por aeronave:**

$$
p_1 + p_{31} = 2x_1
$$

$$
p_2 + p_{32} = 2x_2
$$

$$
p_{33} = 2x_3
$$

---

**3. Limite de cada pool de pilotos:**

$$
p_1 \leq 30
$$

$$
p_2 \leq 20
$$

$$
p_{31} + p_{32} + p_{33} \leq 10
$$

---

**4. Capacidade de manutenção:**

$$
x_1 + \tfrac{3}{4}x_2 + \tfrac{5}{3}x_3 \;\leq\; 40
$$

---

**5. Domínio das variáveis:**

$$
x_j, p_1, p_2, p_{31}, p_{32}, p_{33} \in \mathbb{Z}_{\geq 0}
$$

---

## Observações

- As variáveis $p_1$, $p_2$, $p_{31}$, $p_{32}$, $p_{33}$ foram incluídas para respeitar a **habilitação específica dos pilotos**.  
- Se considerássemos apenas o total de pilotos, poderíamos perder a restrição de compatibilidade.  
- A restrição de manutenção foi convertida para "equivalente a Boeing 717", conforme o enunciado.  
- O modelo busca **maximizar receita**, sujeito a orçamento, pilotos e manutenção.  


In [ ]:
!pip install pulp
import pulp

# Parâmetros
custos = {1: 5.1, 2: 3.6, 3: 6.8}
receitas = {1: 330, 2: 300, 3: 420}
verba = 220
pilotos_pool = {1: 30, 2: 20, 3: 10}
manutencao_max = 40             

# Modelo
modelo = pulp.LpProblem("Compra_Aeronaves", pulp.LpMaximize)

# Variáveis de decisão
aeronaves = {j: pulp.LpVariable(f"aeronaves_{j}", lowBound=0, cat="Integer") for j in [1,2,3]}

# Variáveis de alocação de pilotos
u1 = pulp.LpVariable("u1_pool717_para_717", lowBound=0, cat="Integer")
u2 = pulp.LpVariable("u2_pool737_para_737", lowBound=0, cat="Integer")
v31 = pulp.LpVariable("v31_MD11_para_717", lowBound=0, cat="Integer")
v32 = pulp.LpVariable("v32_MD11_para_737", lowBound=0, cat="Integer")
v33 = pulp.LpVariable("v33_MD11_para_MD11", lowBound=0, cat="Integer")

# Funcao Objetivo
modelo += receitas[1]*aeronaves[1] + receitas[2]*aeronaves[2] + receitas[3]*aeronaves[3], "Maximizar_Receita"

# Restrição de orçamento
modelo += custos[1]*aeronaves[1] + custos[2]*aeronaves[2] + custos[3]*aeronaves[3] <= verba, "Restricao_Orcamento"

# Restrições de pilotos
modelo += u1 + v31 == 2 * aeronaves[1], "Pilotos_para_717"
modelo += u2 + v32 == 2 * aeronaves[2], "Pilotos_para_737"
modelo += v33 == 2 * aeronaves[3], "Pilotos_para_MD11"

# Limites dos pools
modelo += u1 <= pilotos_pool[1], "Capacidade_Pool717"
modelo += u2 <= pilotos_pool[2], "Capacidade_Pool737"
modelo += v31 + v32 + v33 <= pilotos_pool[3], "Capacidade_PoolMD11"

# Restrição de manutenção
modelo += 1.0 * aeronaves[1] + (3.0/4.0) * aeronaves[2] + (5.0/3.0) * aeronaves[3] <= manutencao_max, "Capacidade_Manutencao"

# Solver
modelo.solve()

# Resultados
print("Status:", pulp.LpStatus[modelo.status])
print("Receita total:", pulp.value(modelo.objective))
for j in [1,2,3]:
    print(f"Aeronaves tipo {j}:", int(pulp.value(aeronaves[j])))

print("\nAlocação de pilotos:")
print("u1 (pool 717 usados para 717):", int(pulp.value(u1)))
print("u2 (pool 737 usados para 737):", int(pulp.value(u2)))
print("v31 (MD11 -> 717):", int(pulp.value(v31)))
print("v32 (MD11 -> 737):", int(pulp.value(v32)))
print("v33 (MD11 -> MD11):", int(pulp.value(v33)))


Status: Optimal
Receita total: 10050.0
Aeronaves tipo 1: 15
Aeronaves tipo 2: 10
Aeronaves tipo 3: 5

Alocação de pilotos:
u1 (pool 717 usados para 717): 30
u2 (pool 737 usados para 737): 20
v31 (MD11 -> 717): 0
v32 (MD11 -> 737): 0
v33 (MD11 -> MD11): 10
